In [151]:
import cv2
import numpy as np
from skimage import color, feature, io
from scipy.stats import skew, kurtosis
from scipy import ndimage
# from skimage.feature import greycomatrix, greycoprops
import pandas as pd
import os





In [152]:
def calculate_color_features(image):
    # Convert color spaces
    hsv_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lab_image = cv2.cvtColor(image, cv2.COLOR_RGB2Lab)

    # Prepare to collect features
    channels = [image, hsv_image, lab_image]
    channel_names = ['R', 'G', 'B', 'H', 'S', 'V', 'L*', 'a*', 'b*']
    features = {}

    # Calculate mean, skewness, and kurtosis for each channel
    for idx, channel_set in enumerate(channels):
        for c in range(3):
            channel_data = channel_set[:, :, c].flatten()
            features[f'mean_{channel_names[idx*3 + c]}'] = np.mean(channel_data)
            features[f'skew_{channel_names[idx*3 + c]}'] = skew(channel_data)
            features[f'kurtosis_{channel_names[idx*3 + c]}'] = kurtosis(channel_data)

    return features

In [153]:
import cv2
import numpy as np
import mahotas as mh
from scipy.stats import skew, kurtosis

def calculate_texture_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    glcm = mh.features.haralick(gray_image)
    texture_features = {
        'contrast': glcm.mean(axis=0)[1],
        'correlation': glcm.mean(axis=0)[2],
        'homogeneity': glcm.mean(axis=0)[4],
        'energy': glcm.mean(axis=0)[8]
    }
    return texture_features




In [154]:
def calculate_additional_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    entropy = np.sum(gray_image * np.log2(gray_image + 1))
    intensity = np.mean(gray_image)
    return {'entropy': entropy, 'intensity': intensity}


In [155]:
# # Load the image
# image = cv2.imread('C:/vs code/H2O2/h202/raw data/data cut/100uM new/IMG20240509140325.jpg')
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# # Calculate features
# color_features = calculate_color_features(image)
# texture_features = calculate_texture_features(image)
# additional_features = calculate_additional_features(image)

# # Combine all features
# all_features = {**color_features, **texture_features, **additional_features}
# print(all_features)

In [156]:
# Path to the folder containing images
folder_path = 'C:/vs code/H2O2/h202/raw data/data cut/90uM new'

# List to store all image features
all_image_features = []

# Iterate over each image in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        # Load the image
        image_path = os.path.join(folder_path, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Calculate color features
        color_features = calculate_color_features(image)

        # Calculate texture features
        texture_features = calculate_texture_features(image)

        # Calculate entropy and intensity
        add_ft = calculate_additional_features(image)

        # Combine all features
        all_features = {**color_features, **texture_features, **add_ft}

        # Add filename as a feature
        all_features['filename'] = filename

        # Append features to the list
        all_image_features.append(all_features)

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_image_features)

# Save DataFrame to Excel file
output_file = 'image_features9.xlsx'
df.to_excel(output_file, index=False)

print(f"Features saved to {output_file}")

c:\Python311\Lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Features saved to image_features9.xlsx
